In [18]:
from jupyter_cadquery import set_defaults, set_sidecar
from jupyter_cadquery.cadquery import (PartGroup, Part, show)

set_defaults(theme="dark")
set_sidecar("CadQuery", init=True)
import cadquery as cq

print(f"Imported cadquery version {cq.__version__}")

Imported cadquery version 2.1


In [161]:
CHAMBER_DIA = 95
PADDLE_DIA = CHAMBER_DIA - 3 # A little space to allow for misalignment while mixing
FUNNEL_H = 30
PADDLE_H = 80 # 170 total height of mix container
THICK = 4
SHAFT_DIA = 8
POUR_HOLE_DIA = 12
POUR_HOLE_LEN = 8
TOP_SHAFT_DIA = 8
TOP_SHAFT_LEN = 30
THROUGH_ROD_DIA = 2.5
PLUG_BORE_HOLE_DIA = 3.2
GROMMET_THICK = 3 # mm
nseg = 2
PI=3.14159265

XY = cq.Workplane("XY")


def createFunnel():
    plug_factor = 1.05 # Enlarge plug area so it's a slip fit
    return (XY.circle(CHAMBER_DIA/2).extrude(FUNNEL_H)
            .cut(cq.Solid.makeCone(POUR_HOLE_DIA/2 + GROMMET_THICK, CHAMBER_DIA/2, FUNNEL_H))
            .union(XY.circle(CHAMBER_DIA/2).extrude(-GROMMET_THICK*plug_factor))
            .cut(XY.circle(POUR_HOLE_DIA/2 + GROMMET_THICK*plug_factor).extrude(-GROMMET_THICK*plug_factor))
    )

def createPaddle():
    # A hollow cylinder
    profile = (
        XY.circle(PADDLE_DIA/2).extrude(PADDLE_H) 
        .cut(XY.circle(PADDLE_DIA/2-THICK).extrude(PADDLE_H))
    )

    # Stack a series of struts into the cylinder
    paddle = profile
    HORIZ_BEAM_PITCH = (PADDLE_H-FUNNEL_H)/(nseg-1)
    strut = XY.circle(PADDLE_DIA/2).extrude(THICK)
    for i in range(0, nseg):
        zoffs = i*HORIZ_BEAM_PITCH
        paddle = paddle.union(strut.translate((0,0,FUNNEL_H+zoffs)))

    # Add strut to the bottom to allow for a screw while preventing flexing 
    strut = cq.Solid.makeCone(POUR_HOLE_DIA/2 + GROMMET_THICK, CHAMBER_DIA/2, FUNNEL_H)
    strut = strut.cut(strut.translate((0,0,THICK)))
    paddle = paddle.union(strut)
    
    # Add a center bore to accept a metal shaft
    paddle = (
        paddle.union(XY.circle(SHAFT_DIA/2 + THICK)
                     .extrude(PADDLE_H)) 
        .cut(XY.circle(SHAFT_DIA/2)
                   .extrude(PADDLE_H*1.5))
    )

    # Add back a little material at the bottom
    paddle = paddle.union(XY.circle(POUR_HOLE_DIA/2).extrude(THICK))
    # Cut bottom stopper loose fit hole
    paddle = paddle.cut(XY.circle(PLUG_BORE_HOLE_DIA/2).extrude(2*THICK)) 
    
    # Shave off any extra bits, including what would intersect the funnel
    paddle = paddle.cut(createFunnel()).intersect(XY.circle(PADDLE_DIA/2).extrude(PADDLE_H*1.2)).intersect(createIntersectionProfile())
    
    # Add a ring around the top to help with 3d printing
    paddle = paddle.union(
        XY.circle(PADDLE_DIA/2).extrude(THICK).cut(
            XY.circle(PADDLE_DIA/2-THICK).extrude(THICK)).translate((0,0,PADDLE_H)))
        
    return paddle


        
def createIntersectionProfile(num_arms = 3, twist_amount=90):
    # Intersect with an S-curve shape to help curl the investment down into the hole
    PUNCH_R = PADDLE_DIA/2 * 0.8
    PUNCH_THICK = SHAFT_DIA + 2*THICK
    PUNCH_H = PADDLE_H*1.5
    SHIFT = PUNCH_THICK/2
    punch_segment = (
        XY.moveTo(-SHIFT, 0)
            .radiusArc((2*PUNCH_R - SHIFT, 0), PUNCH_R)
            .lineTo(2*PUNCH_R - PUNCH_THICK - SHIFT, 0)
            .radiusArc((PUNCH_THICK - SHIFT, 0), -(PUNCH_R - PUNCH_THICK))
            .close()
            .twistExtrude(PUNCH_H, twist_amount)
    )
    result = punch_segment
    for i in range(1, num_arms):
        result = result.union(punch_segment.rotate((0,0,0), (0,0,1), i * (360/num_arms)))
    return result

def createPlug():
    # Create the spinning plug at the base
    return (XY.circle(POUR_HOLE_DIA/2 - GROMMET_THICK*0.8).extrude(POUR_HOLE_LEN) 
           .union(XY.circle(POUR_HOLE_DIA/2 + GROMMET_THICK).extrude(-GROMMET_THICK)) 
           .union(XY.circle(POUR_HOLE_DIA/2 - GROMMET_THICK*0.5).extrude(POUR_HOLE_LEN - GROMMET_THICK).translate((0,0,GROMMET_THICK))) 
           .cut(XY.circle(THROUGH_ROD_DIA/2).extrude(2*GROMMET_THICK, both=True)) 
           .edges(">Z").chamfer(GROMMET_THICK*1.4)
    )

funnel = createFunnel()
paddle = createPaddle()
plug = createPlug()

combined = funnel.union(paddle.translate((0,0,0.3)))

pg = PartGroup([
    Part(funnel, show_faces=False),
    Part(paddle),
    Part(createIntersectionProfile(), show_faces=False),
    Part(plug.rotate((0,0,0), (0,1,0), 180).translate((0,0,-GROMMET_THICK)), "plug"),
])
pg


Done, using side car 'Cadquery'


In [162]:
cq.exporters.export(paddle, "vacuum_mixer_paddle.stl")
cq.exporters.export(plug, "vacuum_mixer_plug.stl")
cq.exporters.export(funnel, "vacuum_mixer_funnel.stl")
cq.exporters.export(combined, "vacuum_mixer_paddle_combined.stl")
print("done")

done
